### 1. Import Libraries

In [20]:
import yfinance as yf
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from itertools import combinations

### 2. Data Obtention

In [21]:
# Lista de empresas a evaluar
tickers_list = ['CLF', 'NUE', 'ZEUS', 'STLD', 'RS', 'MTUS', 'FRD', 'X', 'MSB', 'CMC', 'ACNT', 'RDUS']

# Descargar datos históricos
tickers = yf.Tickers(" ".join(tickers_list))
hist = tickers.history(start='2021-01-01',end='2024-12-31')
adj_close = hist['Close'].dropna(axis=1, how='any')

[*********************100%***********************]  12 of 12 completed


### 3. Portfolios Analysis

In [22]:
# Función para calcular métricas del portafolio
def calcular_metricas(adj_close_values):
    R = np.log(adj_close_values[1:] / adj_close_values[:-1])  # Retornos logarítmicos
    RE = np.mean(R, axis=0) * 252  # Retorno esperado anualizado
    RI = np.std(R, axis=0) * np.sqrt(252)  # Riesgo anualizado
    S = np.cov(R, rowvar=False)  # Matriz de covarianza
    corr = np.corrcoef(R, rowvar=False)  # Matriz de correlación
    return RE, RI, S, corr

# Obtener métricas generales
adj_close_values = adj_close.values
RE, RI, S, correlation_matrix = calcular_metricas(adj_close_values)

# Evaluar todas las combinaciones posibles de 4 activos
mejores_portafolios = []
for subset in combinations(adj_close.columns, 4):
    indices = [adj_close.columns.get_loc(ticker) for ticker in subset]
    corr_submatrix = correlation_matrix[np.ix_(indices, indices)]

    # Filtrar combinaciones con correlación > 0.5
    if np.any(np.triu(corr_submatrix, k=1) > 0.5):
        continue  # Saltar esta combinación

    # Extraer datos de la combinación aceptable
    RE_sub = RE[indices]
    S_sub = S[np.ix_(indices, indices)]
    weights = np.ones(4) / 4  # Pesos iniciales iguales

    # Definir restricciones y límites
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1}]
    bounds = [(0, None)] * 4

    # Optimizar el portafolio
    res = minimize(lambda w: w @ S_sub @ w.T, x0=weights, method='SLSQP', bounds=bounds, constraints=constraints, options={'disp': False, 'maxiter': 1000, 'ftol': 1e-12})

    if res.success:
        # Calcular métricas del portafolio optimizado
        ReP = res.x @ RE_sub.T
        varP = res.x @ S_sub @ res.x.T
        RiP = np.sqrt(varP)*np.sqrt(252)
        SharpeP = ReP / RiP

        # Guardar resultados
        mejores_portafolios.append({
            "Activos": subset,
            "ReP (%)": round(ReP * 100, 4),
            "RiP (%)": round(RiP * 100, 4),
            "SharpeP": round(SharpeP, 4)
        })

# Ordenar por el mejor Sharpe Ratio
mejores_portafolios = sorted(mejores_portafolios, key=lambda x: x["SharpeP"], reverse=True)

# Convertir a DataFrame y mostrar los resultados
df_resultados = pd.DataFrame(mejores_portafolios)
print(df_resultados)

# Guardar en un archivo Excel
df_resultados.to_excel("mejores_portafolios.xlsx", index=False)
print("\nResultados guardados en 'mejores_portafolios.xlsx'")

                   Activos  ReP (%)  RiP (%)  SharpeP
0     (ACNT, FRD, MSB, RS)  17.3895  25.0917   0.6930
1   (ACNT, FRD, MSB, STLD)  18.0853  29.2338   0.6186
2    (ACNT, CMC, FRD, MSB)  16.3599  28.2561   0.5790
3    (ACNT, FRD, MSB, NUE)  14.9913  28.9724   0.5174
4   (ACNT, FRD, MSB, MTUS)  13.4665  31.7556   0.4241
5   (ACNT, FRD, MSB, ZEUS)  12.8674  31.7779   0.4049
6      (ACNT, FRD, MSB, X)  12.1779  31.2571   0.3896
7    (ACNT, CLF, FRD, MSB)   8.2876  31.6611   0.2618
8      (FRD, MSB, RDUS, X)   7.4109  34.0704   0.2175
9     (ACNT, FRD, RDUS, X)   6.9768  33.5489   0.2080
10  (ACNT, FRD, MSB, RDUS)   5.9856  31.1392   0.1922
11    (ACNT, MSB, RDUS, X)   4.8278  32.3309   0.1493

Resultados guardados en 'mejores_portafolios.xlsx'
